# Webscrapping Project - Metal Bands

Use both Senlenium and BeautifulSoup to parse http://www.metalstorm.net/home/. 

In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from collections import Counter
from time import sleep
import pandas as pd
import requests
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py

py.init_notebook_mode(connected=True)
%matplotlib inline

In [ ]:
driver = webdriver.Chrome('/Users/max/Downloads/chromedriver')

In [ ]:
action = ActionChains(driver)

In [ ]:
url = 'http://www.metalstorm.net/home/'

In [ ]:
driver.get(url)

In [ ]:
band_dropdown = driver.find_element_by_xpath('//*[@id="main_menu"]/ul/li[3]/a[1]')
action.move_to_element(band_dropdown).perform()
driver.find_element_by_xpath('//*[@id="main_menu"]/ul/li[3]/ul/li[1]/a').click()

In [ ]:
genre_dropdown = {3:'Black Metal',4:'Death Metal',6:'Doom Metal',15:'Heavy Metal',
                  22:'Progress Metal',29:'Thrash Metal'}

## Scrapping the Page

From the band page, extract the band info from the page: genre, country, formed year, split year, albums,avg_score ,last_album year 

In [ ]:
def extract_band_info(band_info,genre):
    
    table = driver.find_element_by_css_selector('div#page-content tbody:nth-of-type(1)')    
    rows = table.find_elements_by_tag_name("tr")
    
    for i in range(0,len(rows)):
        
        table = driver.find_element_by_css_selector('div#page-content tbody:nth-of-type(1)')   
        rows = table.find_elements_by_tag_name("tr")
        
        band_name = rows[i].find_element_by_css_selector('td:nth-child(3) b a').text
        
        band_info[band_name] = [genre]       
        band_info[band_name] += [rows[i].find_element_by_css_selector('td:nth-child(5)').text]   
        band_info[band_name] += [rows[i].find_element_by_css_selector('td:nth-child(6)').text]   
        band_info[band_name] += [rows[i].find_element_by_css_selector('td:nth-child(7)').text]
        
        rows[i].find_element_by_css_selector('td:nth-child(3) a').click()

        sleep(4)
        
        result = requests.get(driver.current_url) 
        soup = BeautifulSoup(result.content)
        table = soup.find('div', id = 'discotab1')
        rows = table.find_all('tr')

        albums = []
        for row in rows:
            albums.append(row.text.strip())
            
        score_cnt = len(albums)
        
        score = 0
        
        band_info[band_name] += [len(albums)]
        for i in range(len(albums)):
            if len(albums[i].split('\n')) ==4:
                score += float(albums[i].split('\n')[-1])
            else:
                score += 0
                score_cnt -= 1
                
        band_info[band_name] += [round(score/score_cnt,2)]     
        band_info[band_name] += [albums[-1].split('\n')[0]]
        
        driver.back()
        
        sleep(4)
        
    pass 

Click on every band to gather all the band info for each genre

In [ ]:
band_info = {}

for i in genre_dropdown.keys():
    
    driver.find_element_by_xpath('//*[@id="filter-styles"]/a[1]').click()
    sleep(1)
    driver.find_element_by_xpath(f'//*[@id="filter-styles"]/ul/li[{i}]/label/input').click()
    sleep(1)
    driver.find_element_by_xpath('//*[@id="page-content"]/form/div/div/div[3]/button').click()
    sleep(5)
    
    last_page = int(driver.find_element_by_xpath('//*[@id="page-content"]/div[3]/div[2]/ul/li[7]/a').text)  
    pages = [2,3,4,5,6,7]+[8]*(last_page-1-6)    
    genre = genre_dropdown[i]
    
    for j in pages:
            
        extract_band_info(band_info,genre)
            
        data = pd.DataFrame.from_dict(band_info,orient='index',\
                            columns=['Genre', 'Country', 'Formed', 'Split','Albums','Avg_Score','Last_album'])
        data.to_csv('band_info.csv',mode='a')
        band_info = {}
            
        driver.find_element_by_xpath(f'//*[@id="page-content"]/div[3]/div[2]/ul/li[{j}]/a').click()
               
        sleep(5)
    
    driver.find_element_by_xpath('//*[@id="filter-styles"]/a[2]').click()    
    driver.find_element_by_xpath('//*[@id="page-content"]/form/div/div/div[3]/button').click()
    
    sleep(8)

Save the data to csv file

In [ ]:
data = pd.DataFrame.from_dict(band_info,orient='index',\
                            columns=['Genre', 'Country', 'Formed', 'Split','Albums','Avg_Score','Last_album'])
data.to_csv('band_info.csv',mode='a')

In [ ]:
band.drop(band[band['Last_album']==' '].index,inplace=True)

## Data Exploration and Visualization

Country coordinates for map plot

In [ ]:
coordinates = {'Norway':[60.4720,8.4689],'USA':[37.0902,-95.7129],
                'Germany':[51.1657,10.4515],'Sweden':[60.1282,18.6435],
                'France': [46.2276,2.2137],'Finland':[61.9241,25.7482],
                'United Kingdom':[55.3781,-3.4360],'Canada':[56.1304,-106.3468],
                'Italy':[41.8719,12.5674],'The Netherlands':[52.1326,5.2913]}

In [ ]:
country_coor = pd.DataFrame.from_dict(coordinates,orient='index',\
                            columns=['Lat', 'Long'])

In [ ]:
black_coor = pd.DataFrame(band[band['Genre']=='Black Metal']['Country'].value_counts().head(5)).\
                merge(country_coor,how='left',left_index=True,right_index=True).rename(columns={'Country':'Bands'})
death_coor = pd.DataFrame(band[band['Genre']=='Death Metal']['Country'].value_counts().head(5)).\
                merge(country_coor,how='left',left_index=True,right_index=True).rename(columns={'Country':'Bands'})
doom_coor = pd.DataFrame(band[band['Genre']=='Doom Metal']['Country'].value_counts().head(5)).\
                merge(country_coor,how='left',left_index=True,right_index=True).rename(columns={'Country':'Bands'})
thrash_coor = pd.DataFrame(band[band['Genre']=='Thrash Metal']['Country'].value_counts().head(5)).\
                merge(country_coor,how='left',left_index=True,right_index=True).rename(columns={'Country':'Bands'})
heavy_coor = pd.DataFrame(band[band['Genre']=='Heavy Metal']['Country'].value_counts().head(5)).\
                merge(country_coor,how='left',left_index=True,right_index=True).rename(columns={'Country':'Bands'})

In [ ]:
black_coor['Genre']='Black Metal'
death_coor['Genre']='Death Metal'
doom_coor['Genre']='Doom Metal'
thrash_coor['Genre']='Thrash Metal'
heavy_coor['Genre']='Heavy Metal'

In [ ]:
pd.concat([black_coor,death_coor,doom_coor,thrash_coor,heavy_coor]).to_csv('coordinate.csv')

In [ ]:
band[band['Genre']=='Black Metal'].groupby('Formed').agg({'Avg_Score':'mean','Formed':'count'})

## Dash

In [ ]:
%%writefile metal.py

import pandas as pd
import numpy as np
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from dash.dependencies import Input, Output

mapbox_access_token = 'pk.eyJ1IjoiY3J5c3RhbG0iLCJhIjoiY2p3cGJ1ZWF3MHZ2MjQwcHJ6ZjdtaHBtZCJ9.ylD2luL9p8CAHg0oZWrReg'
band = pd.read_csv('band.csv',index_col=0)
coordinates = pd.read_csv('coordinate.csv',index_col=0)
genre = band['Genre'].value_counts()
color = {'Black Metal':'Grey','Death Metal':'Red','Thrash Metal':'Blue','Doom Metal':'Purple','Heavy Metal':'Yellow'}
    
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Until The Light Takes Us',style={
                                'textAlign': 'center',
                                'color':'#7f7f7f'}
           ),
    
    html.Div(children='A Century\'s Reign In Chaos', 
             style={
                    'textAlign': 'center'
                    }
            ),
    
    dcc.Dropdown(
        id='genre_select',
        options=[{'label': i, 'value': i} for i in ['Black Metal', 'Death Metal', 'Doom Metal',
       'Thrash Metal', 'Heavy Metal']],
        value=['Black Metal'],
        multi=True
    ),
    
    dcc.Dropdown(
        id='year_select',
        options=[{'label': i, 'value': i} for i in [1985,1990, 1995, 2000]],
        value=1995
    ),
    
    html.Div(
            '3510 Bands Info In Total', id='total-bands', style={'textAlign': 'center'}
    ),
    

    html.Div(               
            dcc.Graph(id='total_bands_genre',
                     figure={'data':[
                                     {'x':genre.index, 
                                      'y':genre.values,
                                      'type':'bar',
                                      'color':'Purple',
                                      'opacity':0.6
                                     }],
                            'layout':go.Layout(
                                        title='Genre',
                                        yaxis={'title':'Band Counts'},
                                        hovermode='closest'
                                        )
                            }
                
            )
    ),
    
    html.Div([dcc.Graph(id='origin_country',
                        className='six columns'),
              dcc.Graph(id='origin_year',
                        className='six columns'),
             ],className='Row'
            ),

    html.Div([
        html.Div([
                dcc.Graph(id='pop_year')
        ],
        className='six columns'),
        html.Div([
                dash_table.DataTable(id='score_table',
                                    columns=[
                                        {"name": i, "id": i} for i in (band.reset_index().rename(columns={'index':'Band'}).columns)
                                    ],
                                    sort_action='native',
                                    filter_action='native'
                                    )]
            ,className='six columns')                
        ],className='Row')
    ])


@app.callback(
    dash.dependencies.Output('origin_country', 'figure'),
    [dash.dependencies.Input('genre_select', 'value')])
def update_country_graph(genres):
    traces = []
    for genre in genres:
        traces.append(go.Scattermapbox(
                        lat=coordinates[coordinates['Genre']==f'{genre}']['Lat'],
                        lon=coordinates[coordinates['Genre']==f'{genre}']['Long'],
                        mode='markers',
                        marker = {
                                'size': coordinates[coordinates['Genre']==f'{genre}']['Bands']/2,
                                'sizemode': 'diameter',
                                'color': color[f'{genre}'],
                                'opacity': 0.6
                        },
                        text = list(zip(coordinates[coordinates['Genre']==f'{genre}'].index,
                                        coordinates[coordinates['Genre']==f'{genre}']['Bands'])),
                        name = f'{genre}'
                        )
                    )
    return {
            'data': traces,
                     
            'layout':go.Layout(
                        title='Origin Map',
                        autosize=True,
                        hovermode='closest',
                        mapbox=dict(
                                accesstoken=mapbox_access_token,
                                bearing=0,
                                center=dict(
                                        lat=45.7465,
                                        lon=-39.4629
                                        ),
                        pitch=0,
                        zoom=1
                        ),
            )
    }


@app.callback(
    dash.dependencies.Output('origin_year', 'figure'),
    [dash.dependencies.Input('genre_select', 'value')])
def update_year_graph(genres):
    traces = []
    for genre in genres:
        traces.append(go.Line(
                        x=band[band['Genre']==f'{genre}']['Formed'].value_counts().sort_index().index, 
                        y=band[band['Genre']==f'{genre}']['Formed'].value_counts().sort_index().values,
                        text=band[band['Genre']==f'{genre}']['Formed'].value_counts().sort_index().index,
                        textposition='top center',
                        name = f'{genre}'
                        )
        )
    return {
            'data': traces,
        
            'layout': go.Layout(
                        title='Bands Formed By Year',
                        xaxis=dict(title='Year',
                                  tickangle=45,
                                  tickfont=dict(family='Rockwell',
                                                size=15
                                            )
                                  ),
                        yaxis=dict(title='Band Counts')
                    )
            }

@app.callback(
    dash.dependencies.Output('pop_year', 'figure'),
    [dash.dependencies.Input('genre_select', 'value'),
     dash.dependencies.Input('year_select', 'value')])
def update_pop_graph(genre_select,year_select):
    pop = band[(band['Formed']==year_select)&(band['Genre'].isin(genre_select))].groupby('Genre').agg({'Genre':'count'})

    return {
            'data': [go.Pie(labels=pop.index,
                           values=pop['Genre'],
                           hoverinfo='label+percent',
                           text=pop['Genre']
                    )],
            'layout':go.Layout(
                            title=f'Popularity in {year_select}',     
                            hovermode='closest'
            )
    }

@app.callback(
    dash.dependencies.Output('score_table', 'data'),
    [dash.dependencies.Input('genre_select', 'value')])
def update_table(genre_select):
    table = []
    for genre in genre_select:
        table.append(band[(band['Genre']==f'{genre}')].sort_values(by='Avg_Score',ascending=False)
                     .head(5).reset_index().rename(columns={'index':'Band'})
        )

    return pd.concat(table).to_dict('records') if table != [] else []


if __name__ == '__main__':
    app.run_server(debug=True)